In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=8eb271f0247903a037dfce0f75ba135397a97e91306c1fee5d1986867aae7044
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
!pip install requests

In [ ]:
!pip install pandas

In [ ]:
#importing the required libraries of python
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        Title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        Title_value = Title.text

        # Title as a string value
        Title_string = Title_value.strip()

    except AttributeError:
        Title_string = ""

    return Title_string

In [ ]:
# Function to extract Product Price
def get_price(soup):

    try:
        Price = soup.find("span", attrs={'class':'a-price a-text-price a-size-base'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            Price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            Price = ""

    return Price


In [ ]:
# Function to extract Product Rating
def get_rating(soup):

    try:
        Rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            Rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            Rating = ""

    return Rating

In [ ]:
# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        Review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        Review_count = ""

    return Review_count

In [ ]:
# Function to extract company of the Laptop
def get_company(soup):
    try:
        Company = soup.find("td", attrs={'class':'a-size-base prodDetAttrValue'}).string.strip()

    except AttributeError:
        Company = ""

    return Company

In [ ]:
# Function to extract Availability Status
def get_availability(soup):
    try:
        Available = soup.find("div", attrs={'id':'availability'})
        Available = Available.find("span").string.strip()

    except AttributeError:
        Available = "Not Available"

    return Available

In [ ]:
if __name__ == '__main__':

    # add your user agent
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=Laptops&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"Company":[],"Title":[], "Price":[], "Rating":[], "Reviews":[],"Availability":[]}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in/" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['Company'].append(get_company(new_soup))
        d['Title'].append(get_title(new_soup))
        d['Price'].append(get_price(new_soup))
        d['Rating'].append(get_rating(new_soup))
        d['Reviews'].append(get_review_count(new_soup))
        d['Availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['Title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['Title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df

,Company,Title,Price,Rating,Reviews,Availability
0,‎ASUS,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...","₹22,990.00",3.9 out of 5 stars,"1,187 ratings",In stock
1,‎Acer,Acer [SmartChoice] One 14 AMD Ryzen 3 3250U Pr...,"₹28,490.00",3.7 out of 5 stars,285 ratings,In stock
2,‎ASUS,"ASUS [SmartChoice] Vivobook 15, Intel Celeron ...","₹26,990.00",3.9 out of 5 stars,"1,187 ratings",In stock
3,‎MSI,"MSI Modern 15, Intel 11th Gen. i3-1115G4, 40CM...","₹29,990.00",3.7 out of 5 stars,56 ratings,In stock
4,‎HP,"HP Laptop 15s, Intel Celeron, 15.6-inch (39.6 ...","₹28,490.00",3.3 out of 5 stars,122 ratings,In stock
5,‎HP,"HP Laptop 15s, 12th Gen Intel Core i3-1215U, 1...","₹39,990.00",4.1 out of 5 stars,"1,965 ratings",In stock
6,‎Lenovo,"Lenovo V15 Intel Core i3 12th Gen 15.6"" (39.62...","₹33,490.00",3.9 out of 5 stars,23 ratings,In stock
7,‎ASUS,"ASUS TUF Gaming F15, 15.6""(39.62 cms) FHD 144H...","₹75,990.00",4.4 out of 5 stars,93 ratings,In stock
8,‎ASUS,"ASUS Vivobook Go 14 (2023), AMD Ryzen 3 7320U,...","₹33,990.00",4.1 out of 5 stars,23 ratings,In stock
9,‎ASUS,"ASUS Vivobook Go 14 (2023), AMD Ryzen 3 7320U,...","₹33,990.00",4.1 out of 5 stars,35 ratings,In stock


In [ ]:
# Load the data from the image.
data = pd.read_csv('/content/amazon_data.csv')

# Clean the data.
data.dropna(inplace=True)

# Extract numerical information from the 'Rating' column.
data['Rating'] = data['Rating'].str.extract('(\d+\.\d+)').astype('float')

# Remove non-numeric characters from the 'Price' column.
data['Price'] = data['Price'].replace('[\₹,]', '', regex=True).astype(float)

In [ ]:
data

,Company,Title,Price,Rating,Reviews,Availability
0,‎ASUS,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",22990.0,3.9,"1,187 ratings",In stock
1,‎Acer,Acer [SmartChoice] One 14 AMD Ryzen 3 3250U Pr...,28490.0,3.7,285 ratings,In stock
2,‎ASUS,"ASUS [SmartChoice] Vivobook 15, Intel Celeron ...",26990.0,3.9,"1,187 ratings",In stock
3,‎MSI,"MSI Modern 15, Intel 11th Gen. i3-1115G4, 40CM...",29990.0,3.7,56 ratings,In stock
4,‎HP,"HP Laptop 15s, Intel Celeron, 15.6-inch (39.6 ...",28490.0,3.3,122 ratings,In stock
5,‎HP,"HP Laptop 15s, 12th Gen Intel Core i3-1215U, 1...",39990.0,4.1,"1,965 ratings",In stock
6,‎Lenovo,"Lenovo V15 Intel Core i3 12th Gen 15.6"" (39.62...",33490.0,3.9,23 ratings,In stock
7,‎ASUS,"ASUS TUF Gaming F15, 15.6""(39.62 cms) FHD 144H...",75990.0,4.4,93 ratings,In stock
8,‎ASUS,"ASUS Vivobook Go 14 (2023), AMD Ryzen 3 7320U,...",33990.0,4.1,23 ratings,In stock
9,‎ASUS,"ASUS Vivobook Go 14 (2023), AMD Ryzen 3 7320U,...",33990.0,4.1,35 ratings,In stock


In [ ]:
# Split the data into training and test sets.
X_train, X_test, y_train, y_test = train_test_split(data['Rating'], data['Price'], test_size=0.25, random_state=42)

In [ ]:
# Train the linear regression model.
model = LinearRegression()
# Reshape X_train and X_test to be 2D arrays, as scikit-learn expects input in this format.
model.fit(X_train.values.reshape(-1, 1), y_train)

LinearRegression()

In [ ]:
# Evaluate the model on the test set.
predictions = model.predict(X_test.values.reshape(-1, 1))
print('Mean squared error:', np.mean((predictions - y_test)**2))

Mean squared error: 101237352.05961967


In [ ]:
# Predict the price of the product.
rating = 4
predicted_price = model.predict(np.array(rating).reshape(-1, 1))[0]
print('Predicted price:', predicted_price)

Predicted price: 39201.293634496906
